In [1]:
import pandas as pd
import numpy as np
import math as m
from collections import Counter

## Generate Dataframe of Keyboard Characters

In [2]:
characters = ""
separated_characters = ""
for i in range(33,123):
    characters += chr(i)
for char in "`.,?-_[]{}<>()|'\"\\/":
    characters = characters.replace(char, '')
for char in characters:
    char += ' '
    separated_characters += char
character_list = separated_characters.split(' ')
character_list.remove('')
print(character_list)

['!', '#', '$', '%', '&', '*', '+', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '^', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [3]:
def give_type(character):
    if character.isnumeric():
        return 'Number'
    elif character.islower():
        return 'Lowercase'
    elif character.isupper():
        return 'Uppercase'
    else:
        return 'Special'

In [4]:
character_type_df = pd.DataFrame(character_list, columns=['Character'])
character_type_df['Character Type'] = character_type_df['Character'].apply(give_type)

In [5]:
character_type_df

,Character,Character Type
0,!,Special
1,#,Special
2,$,Special
3,%,Special
4,&,Special
...,...,...
69,v,Lowercase
70,w,Lowercase
71,x,Lowercase
72,y,Lowercase


In [6]:
character_type_df['Character Type'].value_counts()

Character Type
Lowercase    26
Uppercase    26
Special      12
Number       10
Name: count, dtype: int64

## Generate Matrix with the Layout of Characters on a Keyboard

In [7]:
keyboard_matrix = np.array([[['1','!'], ['2','@'], ['3','#'], ['4','$'], ['5','%'], ['6','^'], ['7','&'], ['8','*'], ['9','?'], ['0','?'], ['?','?'], ['=','+']],
                 [['q','Q'], ['w','W'], ['e','E'], ['r','R'], ['t','T'], ['y','Y'], ['u','U'], ['i','I'], ['o','O'], ['p','P'], ['?','?'], ['?','?']],
                 [['a','A'], ['s','S'], ['d','D'], ['f','F'], ['g','G'], ['h','H'], ['j','J'], ['k','K'], ['l','L'], [';',':'], ['?','?'], ['?','?']],
                 [['z','Z'], ['x','X'], ['c','C'], ['v','V'], ['b','B'], ['n','N'], ['m','M'], ['?','?'], ['?','?'], ['?','?'], ['?','?'], ['?','?']]])

In [8]:
keyboard_matrix[0]

array([['1', '!'],
       ['2', '@'],
       ['3', '#'],
       ['4', '$'],
       ['5', '%'],
       ['6', '^'],
       ['7', '&'],
       ['8', '*'],
       ['9', '?'],
       ['0', '?'],
       ['?', '?'],
       ['=', '+']], dtype='<U1')

In [9]:
keyboard_matrix.shape

(4, 12, 2)

In [10]:
keyboard_matrix[np.where(keyboard_matrix == 'f')][0][0]

'f'

In [11]:
np.where(keyboard_matrix == 'f')[0][0].item()

2

In [12]:
np.where(keyboard_matrix == 'f')[0][0].item()+np.where(keyboard_matrix == 'f')[1][0].item()

5

In [13]:
96*95

9120

## Generate a Pandas Dataframe to Display the Manhattan Distance between Every Possible Pair of Characters

In [14]:
def make_matrix(list, array):
    start_chars = []
    end_chars = []
    distances = []
    for char in list:
        for compchar in list:
            if char == compchar:
                continue
            start_chars.append(char)
            end_chars.append(compchar)
            distances.append(abs(np.where(array == char)[0][0].item() - np.where(array == compchar)[0][0].item()) + abs(np.where(array == char)[1][0].item() - np.where(array == compchar)[1][0].item()))
    return pd.DataFrame({'Start Character': start_chars, 'End Character': end_chars, 'Distance': distances})

In [15]:
character_distance_df = make_matrix(character_list, keyboard_matrix)

In [16]:
character_distance_df

,Start Character,End Character,Distance
0,!,#,2
1,!,$,3
2,!,%,4
3,!,&,6
4,!,*,7
...,...,...,...
5397,z,u,8
5398,z,v,3
5399,z,w,3
5400,z,x,1


In [17]:
character_distance_df[character_distance_df['Distance'] == 0]

,Start Character,End Character,Distance
7,!,1,0
82,#,3,0
156,$,4,0
230,%,5,0
305,&,7,0
...,...,...,...
5079,v,V,0
5153,w,W,0
5227,x,X,0
5301,y,Y,0


In [18]:
character_distance_df[(character_distance_df['Start Character'] == '+')&(character_distance_df['Distance'] == 3)]

,Start Character,End Character,Distance
453,+,9,3
473,+,P,3
500,+,p,3


In [19]:
character_distance_df[character_distance_df['End Character'] == '?']

,Start Character,End Character,Distance


In [20]:
character_distance_df[character_distance_df['Distance'] == 14]

,Start Character,End Character,Distance
483,+,Z,14
510,+,z,14
1432,=,Z,14
1459,=,z,14
3364,Z,+,14
3377,Z,=,14
5335,z,+,14
5348,z,=,14


In [21]:
character_info_df = character_distance_df.merge(character_type_df, left_on='End Character', right_on='Character', how='left')

[OLD APPROACH]  
Approach for selecting first character will be a completely random choice and the next character after this seed will be narrowed down in 2 steps. The first step is narrowing down the options to a specific character type with weighted probabilities based on composition of previously selected password characters. The second step is narrowing down the options to a specific distance from the most recently selected character.

[Current Approach]  
Approach for selecting characters will happen in a sequence where the 4 character types will have an equal chance of being selected and after a character type has been selected it will have its probability of being selected reduced and the percentage it was reduced by will be split evenly between the other 3 character types and this process will continue until the desired number of characters has been selected for the generated sequence of character types

## Generating the Character Type Sequence that the Code Will Follow to Make Random Passowrds

In [22]:
def generate_character_type_sequence(character_length):
    sequence = []
    start_weights = [0.3, 0.3, 0.2, 0.2]
    weight_deltas = [i/2 for i in start_weights]
    character_types = ["Uppercase", "Lowercase", "Number", "Special"]

    char = 0
    while char < character_length:
        while char >= character_length - 2:
            while len(set(sequence)) < 4:
                selected_type = np.random.choice(list({"Uppercase", "Lowercase", "Number", "Special"} - set(sequence))).item()
                sequence.append(selected_type)
                char += 1
            break

        selected_type = np.random.choice(character_types, p=start_weights).item()
        sequence.append(selected_type)

        for type in character_types:
            if type == selected_type:
                if start_weights[character_types.index(type)] - weight_deltas[character_types.index(selected_type)] < 0:
                    start_weights[character_types.index(type)] -= start_weights[character_types.index(type)] 
                else:
                    start_weights[character_types.index(type)] -= weight_deltas[character_types.index(selected_type)]
            else:
                if start_weights[character_types.index(selected_type)] - weight_deltas[character_types.index(selected_type)] < 0:
                    start_weights[character_types.index(type)] += start_weights[character_types.index(selected_type)]/3
                else:
                    start_weights[character_types.index(type)] += weight_deltas[character_types.index(selected_type)]/3
        char += 1
        start_weights = start_weights / np.sum(start_weights, dtype=np.float64)
        weight_deltas = [i/2 for i in start_weights]
    return sequence

In [23]:
print(generate_character_type_sequence(10))

['Lowercase', 'Uppercase', 'Special', 'Lowercase', 'Lowercase', 'Number', 'Number', 'Special', 'Uppercase', 'Uppercase']


In [24]:
print(sorted(generate_character_type_sequence(12)))

['Lowercase', 'Lowercase', 'Lowercase', 'Number', 'Number', 'Special', 'Special', 'Special', 'Uppercase', 'Uppercase', 'Uppercase', 'Uppercase']


In [26]:
def make_random_password(sequence):
    password = ""
    current_chartype = sequence[0]
    current_character = np.random.choice(character_type_df[character_type_df['Character Type'] == current_chartype]['Character'])
    password += current_character
    for i in range(1, len(sequence)):
        filtered_df = character_info_df[(character_info_df['Start Character'] == current_character) & (character_info_df['Character Type'] == sequence[i])]
        current_character = np.random.choice(filtered_df['End Character'])
        password += current_character
    return password

In [39]:
character_type_df

,Character,Character Type
0,!,Special
1,#,Special
2,$,Special
3,%,Special
4,&,Special
...,...,...
69,v,Lowercase
70,w,Lowercase
71,x,Lowercase
72,y,Lowercase


In [40]:
character_distance_df

,Start Character,End Character,Distance
0,!,#,2
1,!,$,3
2,!,%,4
3,!,&,6
4,!,*,7
...,...,...,...
5397,z,u,8
5398,z,v,3
5399,z,w,3
5400,z,x,1


In [41]:
character_info_df

,Start Character,End Character,Distance,Character,Character Type
0,!,#,2,#,Special
1,!,$,3,$,Special
2,!,%,4,%,Special
3,!,&,6,&,Special
4,!,*,7,*,Special
...,...,...,...,...,...
5397,z,u,8,u,Lowercase
5398,z,v,3,v,Lowercase
5399,z,w,3,w,Lowercase
5400,z,x,1,x,Lowercase


In [27]:
make_random_password(generate_character_type_sequence(8))

't63*^tQM'

In [28]:
make_random_password(generate_character_type_sequence(9))

'^8ZI3;9dq'

In [29]:
character_info_df['Character'].value_counts()

Character
#    73
$    73
%    73
&    73
*    73
     ..
w    73
x    73
y    73
z    73
!    73
Name: count, Length: 74, dtype: int64

In [45]:
check = character_info_df.copy().sort_values(by=['Start Character', 'Distance'])

In [46]:
check

,Start Character,End Character,Distance,Character,Character Type
7,!,1,0,1,Number
8,!,2,1,2,Number
19,!,@,1,@,Special
36,!,Q,1,Q,Uppercase
63,!,q,1,q,Lowercase
...,...,...,...,...,...
5365,z,P,11,P,Uppercase
5392,z,p,11,p,Lowercase
5336,z,0,12,0,Number
5335,z,+,14,+,Special


In [47]:
check['Distance'].value_counts()

Distance
3     840
4     808
2     736
5     692
6     568
1     462
7     440
8     328
9     220
10    124
0      72
11     60
12     28
13     16
14      8
Name: count, dtype: int64

In [48]:
check['Distance'].value_counts().keys()[0].item()

3

Equation for probability density function for skew normal distribution:
2/sqrt(2pi)*e^(-(x-E)^2/2w^2)/w

In [49]:
def check_pdf(x, location, scale):
    return 2/m.sqrt(2*m.pi)*m.e**(-((x-location)**2)/(2*scale**2))/scale

In [30]:
def pdf(x, location, scale):
    return 0.8*m.exp(-((x-location)**2)/(2*scale**2))/scale

In [31]:
for i in range(1, 15):
    print(f"x={i}, pdf={pdf(i, 1, 7)}")

x=1, pdf=0.1142857142857143
x=2, pdf=0.11312546324328387
x=3, pdf=0.10971490757548318
x=4, pdf=0.10425760878040517
x=5, pdf=0.09707037903637858
x=6, pdf=0.0885528490152285
x=7, pdf=0.07915077990916544
x=8, pdf=0.06931778968144382
x=9, pdf=0.05948001383093739
x=10, pdf=0.05000739859431648
x=11, pdf=0.04119403298260811
x=12, pdf=0.03324843509098836
x=13, pdf=0.02629329131357818
x=14, pdf=0.02037302623829119


In [52]:
def npdf(x):
    return 0.4*m.exp(-(x**2)/2)

In [53]:
for i in range(-15, 15):
    print(f"x={i}, npdf={npdf(i)}")

x=-15, npdf=5.545373174564683e-50
x=-14, npdf=1.0995140031640859e-43
x=-13, npdf=8.020035127846617e-38
x=-12, npdf=2.1520744640084554e-32
x=-11, npdf=2.1244368998716383e-27
x=-10, npdf=7.714999391855672e-23
x=-9, npdf=1.0307028436619925e-18
x=-8, npdf=5.065666219637671e-15
x=-7, npdf=9.158939382582212e-12
x=-6, npdf=6.091991897885052e-09
x=-5, npdf=1.4906612688314684e-06
x=-4, npdf=0.00013418505116100476
x=-3, npdf=0.004443598615296923
x=-2, npdf=0.054134113294645084
x=-1, npdf=0.2426122638850534
x=0, npdf=0.4
x=1, npdf=0.2426122638850534
x=2, npdf=0.054134113294645084
x=3, npdf=0.004443598615296923
x=4, npdf=0.00013418505116100476
x=5, npdf=1.4906612688314684e-06
x=6, npdf=6.091991897885052e-09
x=7, npdf=9.158939382582212e-12
x=8, npdf=5.065666219637671e-15
x=9, npdf=1.0307028436619925e-18
x=10, npdf=7.714999391855672e-23
x=11, npdf=2.1244368998716383e-27
x=12, npdf=2.1520744640084554e-32
x=13, npdf=8.020035127846617e-38
x=14, npdf=1.0995140031640859e-43


In [54]:
val = 0
for i in range(-4, 4):
    val += npdf(i)
print(val)

1.0025141366411519


In [55]:
val = 0
for i in range(1, 15):
    val += pdf(i, 1, 6)
    # print(val)
print(val)

1.0449696805040292


In [56]:
skew_prob_check = [pdf(i, 1, 7) for i in range(0, 15)]

In [57]:
skew_prob_check

[0.11312546324328387,
 0.1142857142857143,
 0.11312546324328387,
 0.10971490757548318,
 0.10425760878040517,
 0.09707037903637858,
 0.0885528490152285,
 0.07915077990916544,
 0.06931778968144382,
 0.05948001383093739,
 0.05000739859431648,
 0.04119403298260811,
 0.03324843509098836,
 0.02629329131357818,
 0.02037302623829119]

In [58]:
sum(skew_prob_check)

1.1191971528211064

In [59]:
norm_skew_prob_check = skew_prob_check/np.sum(skew_prob_check)

In [60]:
norm_skew_prob_check

array([0.10107733, 0.10211401, 0.10107733, 0.09803001, 0.09315393,
       0.08673215, 0.07912176, 0.07072103, 0.06193528, 0.05314525,
       0.04468149, 0.03680677, 0.0297074 , 0.02349299, 0.01820325])

In [61]:
np.sum(norm_skew_prob_check)

np.float64(0.9999999999999998)

In [63]:
np.random.choice(list(range(0, 15)), p=norm_skew_prob_check).item()

2

In [64]:
func_check = [pdf(i, 1, 7) for i in range(0, 15)]/np.sum([pdf(i, 1, 7) for i in range(0, 15)])
func_check

array([0.10107733, 0.10211401, 0.10107733, 0.09803001, 0.09315393,
       0.08673215, 0.07912176, 0.07072103, 0.06193528, 0.05314525,
       0.04468149, 0.03680677, 0.0297074 , 0.02349299, 0.01820325])

In [32]:
def test_make_random_password(sequence):
    password = ""
    probability_distribution = [test_pdf(i, 1, 7) for i in range(0, 15)]
    # norm_prob_distribution = probability_distribution/np.sum(probability_distribution)
    current_chartype = sequence[0]
    current_character = np.random.choice(character_type_df[character_type_df['Character Type'] == current_chartype]['Character'])
    password += current_character
    for i in range(1, len(sequence)):
        filtered_df = character_info_df[(character_info_df['Start Character'] == current_character) & (character_info_df['Character Type'] == sequence[i])]
        # Attempt set subtraction of filtered_df distances from probability_distribution set and then normalize probability_distribution based on its remaining values
        chosen_distance = np.random.choice(list(set(filtered_df['Distance'])), p=[probability_distribution[i] for i in list(set(filtered_df['Distance']))]/np.sum([probability_distribution[i] for i in list(set(filtered_df['Distance']))])).item()
        last_filtered_df = filtered_df[filtered_df['Distance'] == chosen_distance]
        current_character = np.random.choice(last_filtered_df['End Character'])
        while current_character in password:
            if len(last_filtered_df) > 1 and current_character not in password:
                current_character = np.random.choice(last_filtered_df[last_filtered_df['End Character'] != current_character]['End Character'])
            else:
                chosen_distance = np.random.choice(list(set(filtered_df['Distance'])), p=[probability_distribution[i] for i in list(set(filtered_df['Distance']))]/np.sum([probability_distribution[i] for i in list(set(filtered_df['Distance']))])).item()
                last_filtered_df = filtered_df[filtered_df['Distance'] == chosen_distance]
                current_character = np.random.choice(last_filtered_df['End Character'])
        password += current_character
    return password

In [60]:
test_make_random_password(generate_character_type_sequence(9))

'f&y^3sAHT'

eGF^l!tKN38! / 3-1-4-5-10-5-4-2-6-5-7

In [44]:
test_make_random_password(generate_character_type_sequence(12))

'0L+oUm9@4v$Z'

ci@KzVJ;49f;

In [71]:
test_make_random_password(generate_character_type_sequence(12))

'$RG2Mb4We&*;'

In [56]:
pd = [test_pdf(i, 1, 7) for i in range(0, 15)]
npd = pd/np.sum(pd)

In [57]:
npd

array([0.06184144, 0.13322944, 0.13187687, 0.12790099, 0.1215391 ,
       0.11316053, 0.10323116, 0.04326876, 0.03789343, 0.03251549,
       0.02733717, 0.02251923, 0.01817567, 0.01437355, 0.01113717])

In [ ]:
len(npd)

15

In [ ]:
character_info_df['Distance'].value_counts()/len(character_info_df)

Distance
3     0.155498
4     0.149574
2     0.136246
5     0.128101
6     0.105146
1     0.085524
7     0.081451
8     0.060718
9     0.040726
10    0.022954
0     0.013328
11    0.011107
12    0.005183
13    0.002962
14    0.001481
Name: count, dtype: float64

In [ ]:
set(character_info_df['Distance'])

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}

In [ ]:
for i in range(0, 15):
    print(f"x={i}, pdf={pdf(i, 1, 7)}")
print(sum([pdf(j, 1, 7) for j in range(0, 15)]))

x=0, pdf=0.11312546324328387
x=1, pdf=0.1142857142857143
x=2, pdf=0.11312546324328387
x=3, pdf=0.10971490757548318
x=4, pdf=0.10425760878040517
x=5, pdf=0.09707037903637858
x=6, pdf=0.0885528490152285
x=7, pdf=0.07915077990916544
x=8, pdf=0.06931778968144382
x=9, pdf=0.05948001383093739
x=10, pdf=0.05000739859431648
x=11, pdf=0.04119403298260811
x=12, pdf=0.03324843509098836
x=13, pdf=0.02629329131357818
x=14, pdf=0.02037302623829119
1.1191971528211064


In [ ]:
pdf(0,1,7)/15

0.007541697549552259

Will attempt to implement the pdf function in the same way as the test_pdf function to account for what should be a very low probability of getting a zero.

In [55]:
def test_pdf(x, location, scale):
    return 0.4*m.exp(-((x-location)**2)/(2*scale**2))/scale if x == 0 or x > 6 else 0.8*m.exp(-((x-location)**2)/(2*scale**2))/scale + 0.053*m.exp(-((x-location)**2)/(2*scale**2))/scale

In [39]:
for i in range(0, 15):
    print(f"x={i}, pdf={test_pdf(i, 1, 7)}")
print(sum([test_pdf(j, 1, 7) for j in range(0, 15)]))

x=0, pdf=0.007494561939867556
x=1, pdf=0.12185714285714287
x=2, pdf=0.12062002518315143
x=3, pdf=0.11698352020235893
x=4, pdf=0.11116467536210702
x=5, pdf=0.10350129164753866
x=6, pdf=0.0944194752624874
x=7, pdf=0.08439451907814766
x=8, pdf=0.07391009324783948
x=9, pdf=0.063420564747237
x=10, pdf=0.05332038875118995
x=11, pdf=0.0439231376677059
x=12, pdf=0.03545114391576634
x=13, pdf=0.028035221863102734
x=14, pdf=0.021722739226577983
1.080218500952221


In [61]:
for i in range(0, 15):
    print(f"x={i}, pdf={test_pdf(i, 1, 7)}")
print(sum([test_pdf(j, 1, 7) for j in range(0, 15)]))

x=0, pdf=0.05656273162164194
x=1, pdf=0.12185714285714287
x=2, pdf=0.12062002518315143
x=3, pdf=0.11698352020235893
x=4, pdf=0.11116467536210702
x=5, pdf=0.10350129164753866
x=6, pdf=0.0944194752624874
x=7, pdf=0.03957538995458272
x=8, pdf=0.03465889484072191
x=9, pdf=0.029740006915468693
x=10, pdf=0.02500369929715824
x=11, pdf=0.020597016491304056
x=12, pdf=0.01662421754549418
x=13, pdf=0.01314664565678909
x=14, pdf=0.010186513119145596
0.9146412459570927


### Code to test out implementation of dynamic alteration of probabilty density for password generation based on selected keyword

In [21]:
def test_pdf(x, location, scale, concentration = 'standard'):
    if concentration == 'standard':
        return 0.8*m.exp(-((x-location)**2)/(2*scale**2))/scale if x == 0 or x > 6 else 0.8*m.exp(-((x-location)**2)/(2*scale**2))/scale
    elif concentration == 'dense':
        return 0.4*m.exp(-((x-location)**2)/(2*scale**2))/scale if x > 6 else (0.8*m.exp(-((x-location)**2)/(2*scale**2))/scale if x == 0 else 1.2*m.exp(-((x-location)**2)/(2*scale**2))/scale)
    elif concentration == 'sparse':
        return 1.5*m.exp(-((x-location)**2)/(2*scale**2))/scale if x > 6 else 0.4*m.exp(-((x-location)**2)/(2*scale**2))/scale 

In [11]:
standard_npd = [test_pdf(i, 1, 7, 'standard') for i in range(0, 15)]/np.sum([test_pdf(i, 1, 7, 'standard') for i in range(0, 15)])
dense_npd = [test_pdf(i, 1, 7, 'dense') for i in range(0, 15)]/np.sum([test_pdf(i, 1, 7, 'dense') for i in range(0, 15)])
sparse_npd = [test_pdf(i, 1, 7, 'sparse') for i in range(0, 15)]/np.sum([test_pdf(i, 1, 7, 'sparse') for i in range(0, 15)])
for i in range(0, 15):
    print(f"x={i}, standard={standard_npd[i]}, dense={dense_npd[i]}, sparse={sparse_npd[i]}")
print(np.sum(standard_npd), np.sum(dense_npd), np.sum(sparse_npd))

x=0, standard=0.06184143987783312, dense=0.09334455846509038, sparse=0.05114514746708503
x=1, standard=0.13322944203071654, dense=0.13852323393135135, sparse=0.05166970850721377
x=2, standard=0.13187687053947914, dense=0.13711691882386662, sparse=0.05114514746708503
x=3, standard=0.127900989288916, dense=0.13298305831855842, sparse=0.04960320131655976
x=4, standard=0.12153910164611349, dense=0.12636838488934954, sparse=0.04713590223424833
x=5, standard=0.11316053382136021, dense=0.11765689970178562, sparse=0.04388648415807631
x=6, standard=0.10323115831463033, dense=0.10733298641996117, sparse=0.04003562409087647
x=7, standard=0.04326875715425505, dense=0.030626318439123546, sparse=0.13893469993631272
x=8, standard=0.03789343088770764, dense=0.02682157652415323, sparse=0.12167468622156465
x=9, standard=0.032515488501011505, dense=0.023014982877494285, sparse=0.10440627222236738
x=10, standard=0.027337165700409706, dense=0.01934968316697998, sparse=0.08777883081215745
x=11, standard=0.0

In [22]:
standard_npd = [test_pdf(i, 1, 7, 'standard') for i in range(0, 15)]/np.sum([test_pdf(i, 1, 7, 'standard') for i in range(0, 15)])
dense_npd = [test_pdf(i, 1, 7, 'dense') for i in range(0, 15)]/np.sum([test_pdf(i, 1, 7, 'dense') for i in range(0, 15)])
sparse_npd = [test_pdf(i, 1, 7, 'sparse') for i in range(0, 15)]/np.sum([test_pdf(i, 1, 7, 'sparse') for i in range(0, 15)])
for i in range(0, 15):
    print(f"x={i}, standard={standard_npd[i]}, dense={dense_npd[i]}, sparse={sparse_npd[i]}")
print(np.sum(standard_npd), np.sum(dense_npd), np.sum(sparse_npd))
print(np.sum(standard_npd[:7]), np.sum(dense_npd[:7]), np.sum(sparse_npd[:7]))

x=0, standard=0.10107733294186277, dense=0.09099771094225366, sparse=0.05233352197346053
x=1, standard=0.10211401449480083, dense=0.13789651898753005, sparse=0.05287027136376517
x=2, standard=0.10107733294186277, dense=0.13649656641338048, sparse=0.05233352197346053
x=3, standard=0.09803000954651293, dense=0.13238140856235506, sparse=0.050755748191454335
x=4, standard=0.0931539260242112, dense=0.12579666162683917, sparse=0.04823112059462626
x=5, standard=0.086732153304445, dense=0.11712458945177127, sparse=0.04490620120907267
x=6, standard=0.07912176044409833, dense=0.10684738422424796, sparse=0.040965865127859484
x=7, standard=0.07072103401054396, dense=0.031834299655147634, sparse=0.13731123040855597
x=8, standard=0.061935280577436955, dense=0.027879488877860764, sparse=0.12025290213543914
x=9, standard=0.05314525120172881, dense=0.023922753331797855, sparse=0.10318627173625852
x=10, standard=0.044681491967939004, dense=0.02011288472018626, sparse=0.08675312408035485
x=11, standard=0